# 基础知识

## 面向对象的javascript

关注对象的行为，而不关注对象本身

In [1]:
{
    var duck = {
        duckSinging: function (){
            console.log('嘎嘎嘎');
        }
    }
    
    var chicken = {
        duckSinging: function(){
            console.log('嘎嘎嘎');
        }
    }
    
    var choir = [];
    
    var joinChoir = function (animal){
        if(animal && typeof animal.duckSinging === 'function') {
            choir.push(animal);
            console.log('加个合唱团');
            console.log('合唱团人数：',choir.length);
        }
    };
    
    joinChoir(duck);
    joinChoir(chicken);
}

加个合唱团
合唱团人数： 1
加个合唱团
合唱团人数： 2


In [3]:
{
    var makeSound = function (animal){
        if(animal instanceof Duck){
            console.log('嘎嘎嘎')
        }else if(animal instanceof Chicken){
            console.log('咯咯咯')
        }
    }
    
    var Duck = function(){}
    var Chicken = function(){}
    
    makeSound(new Duck())
    makeSound(new Chicken())
}

嘎嘎嘎
咯咯咯


多态背后的思想是将“做什么”和“谁去做以及怎样去做”分离开来。也就是将“不变的事物”和“可能改变的食物”分离开来。

In [5]:
{
    var makeSound = function(animal){
        animal.sound();
    }
    
    var Duck = function(){}
    
    Duck.prototype.sound = function(){
        console.log('嘎嘎嘎')
    }
    
    var Chicken = function(){}
    
    Chicken.prototype.sound = function(){
        console.log('咯咯咯')
    }
    
    /*附加*/
    var Dog = function(){}
    Dog.prototype.sound = function(){
        console.log('汪汪汪')
    }
    
    makeSound(new Duck)
    makeSound(new Chicken)
    makeSound(new Dog)
}

嘎嘎嘎
咯咯咯
汪汪汪


使用继承来得到多态效果，是让对象表现出多态性的最常用手段。

继承通常包括实现继承和接口继承

封装

In [6]:
{
    var myObject = (function(){
        var __name = 'sven';
        return {
            getName: function(){
                return __name;
            }
        }
    })()
    
    console.log('func getName->',myObject.getName());
    console.log('filed __name->',myObject.__name);
}

func getName-> sven
filed __name-> undefined


封装在更重要的层面体现为*封装变化*

In [ ]:
考虑你的设计中哪些地方可能变化，这种方式与关注会导致重新设计的原因相反。它不是考虑什么时候会迫使你的设计改变，而是考虑你怎样才能够
在不重新设计的情况下进行改变。这里的关键在于封装发生变化的概念，这是许多设计模式的主题。

- 创建型模式 封装创建对象的变化
- 结构型模式 封装对象之间的组合关系
- 行为型模式 封装对象的行为变化

In [13]:
(function (){
    var Plane = function (){
        this.blood = 100;
        this.attackLevel = 1;
        this.defenseLevel = 1;
    }
    
    var plane = new Plane()
    plane.blood = 500;
    plane.attackLevel = 10;
    plane.defenseLevel = 7;
    
    var clonePlane = Object.create(plane);
    console.log('blood',clonePlane.blood)
    console.log('attackLevel',clonePlane.attackLevel)
    console.log('defenseLevel',clonePlane.defenseLevel)

})()

blood 500
attackLevel 10
defenseLevel 7


In [15]:
/*测试 Object.create*/

(function(){
    var origin = {
        b:1,
        c:2
    }
    
    var test = Object.create(origin);
    
    console.log(test === origin)
    console.log(test)
    console.log(origin)
})()

false
{}
{ b: 1, c: 2 }


In [ ]:
/*模拟create*/
(function(){
    Object.create = Object.create || function(obj){
        var F = function(){}
        F.prototype = obj
        return new F;
    }
})()

原型编程的基本规则

- 所有的数据都是对象
- 要得到一个对象，不是通过实例化类，而是找到一个对象作为原型并克隆它
- 对象会记住它的原型
- 如果对象无法响应某个请求，它会把这个请求委托给它自己的原型

In [16]:
{
    var obj1 = new Object()
    var obj2 = {}
    
    console.log(Object.getPrototypeOf(obj1) === Object.prototype)
    console.log(Object.getPrototypeOf(obj2) === Object.prototype)
}

true
true


In [19]:
(function(){
    /*
        在这里Person不是类，而是函数构造器，javascript的函数既可以当普通函数调用，还可以作为构造器调用
        当使用new 运算符来调用函数时，此时的函数就是一个构造器
        
        new运算符来创建对象的过程，实际上也是先克隆 Object.prototype对象，再进行一些其他额外的操作。
    */
    function Person(name){
        this.name = name
    }
    
    Person.prototype.getName = function(){
        return this.name
    }
    
    var a = new Person('sven');
    
    console.log(a.name)
    console.log(a.getName())
    console.log(Object.getPrototypeOf(a) === Person.prototype)
})()

sven
sven
true


In [21]:
(function(){
    function Test1(){
        this.test =1;
        return Number(1)
    }
    function Test2(){
        this.test = 2;
        return {}
    }
    
    var t1 = new Test1,
        t2 = new Test2;
    
    console.log(t1)
    console.log(t2)
})()
/*
    当函数作为构造函数时，如果函数内return 一个对象，则返回该对象，若为其他基本类型则返回 this.
*/

Test1 { test: 1 }
{}


In [22]:
/*
    简单模拟new运算的过程
*/
(function(){
    function Person(name){
        this.name = name;
    }
    Person.prototype.getName = function(){
        return this.name;
    }
    
    var objectFactory = function(){
        var obj = new Object(),
            Constructor = [].shift.call(arguments);
        obj.__proto__ = Constructor.prototype;
        var ret = Constructor.apply(obj, arguments);
        
        return typeof ret === 'object' ? ret : obj;
    }
    
    var a = objectFactory(Person, 'seven');
    
    console.log(a.name)
    console.log(a.getName())
    console.log(Object.getPrototypeOf(a) === Person.prototype)
})()

15:13 - Property '__proto__' does not exist on type 'Object'.


In [23]:
(function(){
    var obj = {name:'sven'}
    var A = function(){}
    A.prototype = obj
    
    var a = new A
    console.log(a.name)
})()

sven


In [ ]:
上述代码执行的过程
- 尝试遍历对象a中的所有属性，但没有找到name 这个属性
- 查找name 属性的这个请求被委托给对对象a 的构造器的原型，他被a.__proto__ 记录着并且指向A.prototype，而A.prototype 被设置为对象obj
- 在对象obj 中找到了 name 属性，并返回它的值

In [24]:
(function (){
    var A = function(){}
    A.prototype = {name:'sven'}
    
    var B = function(){}
    B.prototype = new A();
    
    var b = new B;
    console.log(b.name)
})()

sven


In [25]:
(function(){
    class Animal {
        constructor(name){
            this.name = name;
        }
        getName(){
            return this.name;
        }
    }
    
    class Dog extends Animal {
        constructor(name){
            super(name);
        }
        speak(){
            return "woof";
        }
    }
    
    var dog = new Dog('Scamp');
    console.log(dog.getName(),' says ', dog.speak())
})()

Scamp  says  woof


## this, call , apply

javascript 的this 总是指向一个对象(运行时基于函数的执行环境动态绑定，而非函数被声明时的环境)

- 作为对象的方法调用(指向该对象)
- 作为普通函数调用(指向window)
- 构造器调用(指向返回的对象)
- call ,apply ,bind 调用(指向调用的第一个参数)

## 闭包和高阶函数

闭包的形成与变量的作用域以及变量的生存周期密切相关。

对于全局变量来说，其声明周期是永久的，除非主动销毁
函数内部声明的变量，会在函数调用结束后被销毁

闭包的更多作用

- 封装变量(提炼函数是代码重构的一种常见手段)
- 延续局部变量的寿命

In [ ]:
(function(){
    var mult = (function(){
        var cache = {}
        var calculate = function(){
            var a = 1;
            for(var i =0,l = arguments.length;i<l;i++){
                a = a* arguments[i];
            }
            return a;
        }
        
        return function(){
            var args = Array.prototype.join.call(arguments,',');
            if(args in cache){
                return cache[args];
            }else{
                return cache[args] = calculate.apply(null, arguments)
            }
        }
    })();
})()

### 闭包和内存泄漏

如果闭包的作用域链中保存着一些DOM节点，这个时候就有可能造成内存泄漏。
主要原因是DOM节点由于引用无法被销毁。

### 高阶函数

- 函数可以作为参数被传递
- 函数可以作为返回值输出

AOP(面向切面编程)的主要作用是把一些跟核心业务逻辑模块无关的功能抽离出来。

In [26]:
(function(){
   Function.prototype.before = function(beforefn){
       var _self = this;
       return function(){
           beforefn.apply(this,arguments);
           return _self.apply(this,arguments)
       }
   } 
    
    Function.prototype.after = function(afterfn){
        var _self = this;
        return function(){
            var ret = _self.apply(this,arguments)
            afterfn.apply(this,arguments)
            return ret;
        }
    }
    
    var func = function(){
        console.log(2);
    }
    
    func = func.after(function(){
        console.log(3)
    }).before(function(){
        console.log(1)
    })
    
    func()
})()

2:23 - Property 'before' does not exist on type 'Function'.
10:24 - Property 'after' does not exist on type 'Function'.
23:17 - Property 'after' does not exist on type '() => void'.


In [29]:
(function(){
    var currying = function curry(fn){
        var args = [];
        return function inner(){
            if(arguments.length === 0){
                return fn.apply(this,args)
            }else{
                [].push.apply(args,arguments);
                return inner;
            }
        }
    }
    
    var cost = (function(){
        var money = 0;
        return function(){
            for(var i =0,l= arguments.length;i<l;i++){
                money += arguments[i]
            }
            
            return money
        }
    })()
    
    var _cost = currying(cost)
    _cost(100)
    _cost(200)
    _cost(300)
    console.log(_cost());
})()

600


In [ ]:
(function(){
   Function.prototype.uncurrying = function(){
       var self = this;
       return function(){
           var obj = Array.prototype.shift.call(arguments);
           return self.apply(obj,arguments);
       }
   } 
   
   var push = Array.prototype.push.uncurrying();
   (function(){
       push(arguments,4);
       console.log(arguments)
   })(1,2,3)
})()

In [ ]:
{
    //节流函数
    var throttle = function(fn,interval){
        var _self = fn,
            timer,
            firstTime = true;
        
        return function(){
            var args = arguments,
                _me = this;
            
            if(firstTime){
                _self.apply(_me, args);
                return firstTime = false;
            }
            
            if(timer){
                return false;
            }
            
            timer = setTimeout(function(){
                clearTimeout(timer);
                timer = null;
                _self.apply(_me, args);
            },interval || 500)
        }
    }
}

In [ ]:
{
    //分时函数
    var timeChunk = function(ary, fn, count){
        var obj,
            t;
        
        var len = ary.length;
        var start = function(){
            for(var i =0;i<Math.min(count || 1, ary.length); i++){
                var obj = ary.shift();
                fn(obj);
            }
        }
        
        return function(){
            t = setInterval(function(){
                if(ary.length === 0){
                    return clearInterval(t);
                }
                start()
            },200)
        }
    }
}

In [ ]:
{
    //惰性加载
    var addEvent = (function(){
        if(window.addEventListener){
            return function(elem,type,handler){
                elem.addEventListener(type,handler,false)
            }
        }
        if(window.attachEvent){
            return function(elem,type,handler){
                ele.attachEvent('on'+type,handler)
            }
        }
    })()
    
    var addEvent2 = function(elem,type,handler){
        if(window.addEventListener){
            addEvent = function(elem,type,handler){
               elem.addEventListener(type,handler,false)
            }
        }
        if(window.attachEvent){
            addEvent = function(elem,type,handler){
                ele.attachEvent('on'+type,handler)
            }
        }
        
        addEvent(elem,type,handler)
    }
}